### Compare velocity field in case of naive and Bayes estimated distances   
If a particular bin doesn't have data, then the plot isn't displayed nicely.  
Used x limits of: -10 - 0 [kpc]

In [ ]:
from matplotlib.ticker import FormatStrFormatter
from matplotlib.ticker import StrMethodFormatter
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.cm as cm
import numpy as np
import mpl_scatter_density
import astropy
import pandas as pd
import sys

sys.path.append("../gaia_tools/")
import data_analysis
import transformation_constants
import covariance_generation
from import_functions import import_data

In [ ]:
path = "/hdfs/local/sven/gaia_tools_data/gaia_rv_data_bayes.csv"
data_icrs = import_data(path = path, debug = False)

In [ ]:
def Velocity_Field_Imshow(bin_collection, 
                          title_string = "",
                          radii = [0],
                          display_arrows = False, 
                          plot_circles=False,
                          is_interpolate=False,
                          save = False):

    # Gets the mean of the velocity component inside bins
    H = bin_collection.CalculateValues('v_x')
    H2 = bin_collection.CalculateValues('v_y')

    # Gives the hypotenuse of vectors
    M = np.hypot(H.T, H2.T)

    # Gets the vector coordinates
    VEC_XX, VEC_YY = data_analysis.generate_vector_mesh(bin_collection.bin_boundaries[0], 
                                                        bin_collection.bin_boundaries[1])

    BIN_X_EDGES = bin_collection.bin_boundaries[0][0]
    BIN_Y_EDGES = bin_collection.bin_boundaries[1].T[0]

    extent = [BIN_X_EDGES[0], BIN_X_EDGES[-1], BIN_Y_EDGES[0], BIN_Y_EDGES[-1]]

    fig, ax = plt.subplots(figsize = (10,10))
    ax.set_xlabel(r'$x$ [{0:latex_inline}]'.format(astropy.units.core.Unit('pc')), fontdict={'fontsize': 15}, labelpad = 5)
    ax.set_ylabel(r'$y$ [{0:latex_inline}]'.format(astropy.units.core.Unit('pc')), fontdict={'fontsize': 15}, labelpad = 5)

    if(is_interpolate):
        interpolation = 'Gaussian'
    else:
        interpolation = 'None'

    c = plt.imshow(M, extent = extent, interpolation = interpolation, cmap='jet')

    if(display_arrows):
        # The quiver plot with normalised vector lengths
        q = ax.quiver(VEC_XX, VEC_YY, H.T, H2.T, M, cmap=plt.cm.magma_r)

    # Plot Galaxy center and Sun position
    #ax.plot(0, 0, "x", markersize=20, color='red')
    ax.plot(-8178, 0, "*", markersize=20, color='red')

    cbar = plt.colorbar(c, ax=ax, pad = 0.05)
    cbar.set_label(label ='Velocity in bin [km/s]', labelpad= 30, size = 15)

    XX = bin_collection.bin_boundaries[0]
    YY = bin_collection.bin_boundaries[1]

    plt.xticks(XX[0], rotation=45)
    plt.yticks(YY.T[0])
    plt.ticklabel_format(axis="x", style="sci", scilimits=(3,3))
    plt.ticklabel_format(axis="y", style="sci", scilimits=(3,3))

    plt.title(title_string, pad = 10, fontdict={'fontsize': 20})

    # Plot concentric circles
    if(plot_circles):
        angle = np.linspace( 0 , 2 * np.pi , 150 ) 

        colors=["blue", "green", "white"]
        for i, radius in enumerate(radii):

            radius = radii[i]
            x = radius * np.cos(angle)
            y = radius * np.sin(angle)

            idx = np.where( (x <= XX[0][-1]) &
                            (x >= XX[0][0]) &
                            (y <= YY.T[0][-1]) & 
                            (y >= YY.T[0][0]))[0][:]
            x = x[idx]
            y = y[idx]

            ax.plot(x, y , color=colors[i], label='{0} pc'.format(radius)) 

    plt.legend(loc='upper left')
    plt.grid()
    current_cmap = mpl.cm.get_cmap('jet')
    current_cmap.set_bad(color='grey')
    
    if(save):
        plt.savefig("../out/velocity_field_plots/" +"Velocity_Field_{}_Bayes_Est".format(title_string[1])+'.png', dpi=300)


In [ ]:
# Transform coordinates
galcen_data = data_analysis.get_transformed_data(data_icrs, 
                                       include_cylindrical = True,
                                       debug = True,
                                       is_bayes = True, 
                                       is_source_included = True)

# Generate covariance information
cov_df = covariance_generation.generate_covmatrices(df = data_icrs, 
                                        df_crt = galcen_data, 
                                        transform_to_galcen = True, 
                                        transform_to_cylindrical = True,
                                        is_bayes = True,
                                        debug=True)

galcen_data['cov_mat'] = cov_df['cov_mat']

# Default bins in range: -10 - 10 [kpc]
bin_collection = data_analysis.bin_data(galcen_data,
                                        BL_x = (-10000, 0),
                                        BL_z = (-1000, 1000),
                                        debug = True)

In [ ]:
Velocity_Field_Imshow(bin_collection,
                        title_string="Baseline Velocity Field\n$R_0 = {}$ pc\nBayes Est.".format(transformation_constants.R_0),
                        radii=[5000, 8000],
                        display_arrows=True, 
                        plot_circles=True, 
                        save=False)

### Using naive distance estimation with parallax

In [ ]:
# Transform coordinates
galcen_data = data_analysis.get_transformed_data(data_icrs,
                                        include_cylindrical = True,
                                        debug = True,
                                        is_bayes = False, 
                                        is_source_included = True)

# Generate covariance information
cov_df = covariance_generation.generate_covmatrices(df = data_icrs, 
                                        df_crt = galcen_data, 
                                        transform_to_galcen = True, 
                                        transform_to_cylindrical = True,
                                        is_bayes = False,
                                        debug=True)

galcen_data['cov_mat'] = cov_df['cov_mat']

# Default bins in range: -10 - 10 [kpc]
bin_collection = data_analysis.bin_data(galcen_data,
                                        BL_x = (-10000, 0),
                                        BL_z = (-1000, 1000),
                                        debug = True)

In [ ]:
Velocity_Field_Imshow(bin_collection,
                        title_string="Baseline Velocity Field\n$R_0 = {}$ pc\nNon-Bayes Est.".format(transformation_constants.R_0),
                        radii=[5000, 8000],
                        display_arrows=True, 
                        plot_circles=True, 
                        save=False)

---

## Fields with varying $R_0$

In [ ]:
R_0 = 12000

# Transform coordinates
galcen_data = data_analysis.get_transformed_data(data_icrs,
                                        r_0 = R_0, 
                                        include_cylindrical = True,
                                        debug = True,
                                        is_bayes = True, 
                                        is_source_included = True)

# Default bins in range: -10 - 10 [kpc]
bin_collection = data_analysis.bin_data(galcen_data,
                                        BL_x = (-12000, 0),
                                        BL_y = (-12000, 12000),
                                        BL_z = (-1000, 1000),
                                        N_bins = (10, 10),
                                        debug = True)

Velocity_Field_Imshow(bin_collection,
                        title_string="Baseline Velocity Field\n$R_0 = {}$ pc\nBayes Est.".format(R_0),
                        radii=[5000, 8000],
                        display_arrows=True, 
                        plot_circles=True, 
                        save=False)

---

## Slice of the velocity field

$r \in [5, 12]$ kpc  
$z \in [-200, 200]$ pc

In [ ]:
# Transform coordinates
galcen_data = data_analysis.get_transformed_data(data_icrs,
                                        include_cylindrical = True,
                                        debug = True,
                                        is_bayes = True, 
                                        is_source_included = True)

# Generate covariance information
cov_df = covariance_generation.generate_covmatrices(df = data_icrs, 
                                        df_crt = galcen_data, 
                                        transform_to_galcen = True, 
                                        transform_to_cylindrical = True,
                                        is_bayes = True,
                                        debug=True)

galcen_data['cov_mat'] = cov_df['cov_mat']

galcen_data = galcen_data[(galcen_data.r < 12000) & (galcen_data.r > 5000) ]
galcen_data.reset_index(inplace=True, drop=True)

In [ ]:
# Default bins in range: -10 - 10 [kpc]
bin_collection = data_analysis.bin_data(galcen_data,
                                        BL_x = (-12000, -5000),
                                        BL_y = (-4000, 4000),
                                        BL_z = (-200, 200),
                                        N_bins = (15, 15),
                                        debug = True)

In [ ]:
Velocity_Field_Imshow(bin_collection,
                        title_string="Baseline Velocity Field\n$R_0 = {}$ pc\nBayes Est.".format(transformation_constants.R_0),
                        radii=[6000, 8000, 10000],
                        display_arrows=True, 
                        plot_circles=True,
                        is_interpolate=True,
                        save=False)

plt.title("Baseline Velocity Field\n$R_0 = {}$ pc\n$r \in [5, 12]$ kpc\n$z \in [-200, 200]$ pc".format(transformation_constants.R_0))